In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import re
import time

In [ ]:
def urlBuilder(checkIn, checkOut, adults):
    '''
    Generate URL based on checkin/out days and no. of adults
    ''' 
    url = "https://www.agoda.com/search?city=4064&tick=637383916004"
    url += "&checkIn=" + checkIn
    url += "&checkOut=" + checkOut 
    url += "&rooms=1" 
    url += "&adults=" + str(adults)
    
    return url

def scrollToBtm():
    '''
    Slowly scroll using cmd PgDown to load cards
    ''' 
    SCROLL_PAUSE_TIME = 1.5
    html = driver.find_element_by_css_selector('html')
    # Get scroll height
    time.sleep(4)
    last_height = driver.execute_script("return document.body.scrollHeight")
    

    while True:
        # Scroll down a little
        html.send_keys(Keys.PAGE_DOWN)

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
def scrollpgDwn(no_of_times):
    '''
    Slowly scroll using cmd PgDown to load cards
    ''' 
    SCROLL_PAUSE_TIME = 0.5
    html = driver.find_element_by_css_selector('html')
    # Get scroll height
    time.sleep(4)

    for i in range(no_of_times):
        # Scroll down a little
        html.send_keys(Keys.PAGE_DOWN)
        time.sleep(SCROLL_PAUSE_TIME)

 
        
def nextPage():
    '''
    Click on next page in AGODA
    ''' 
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        next_button = driver.find_element_by_id("paginationNext")
        next_button.click()
        return True
    except NoSuchElementException:
        return False
        
def extractHotelData():
    '''
    Extract hotel data from single page to hotels dict
    '''    
    hotels = {}
    count = 0
    propertyCards = driver.find_elements_by_xpath('//li[@data-selenium="hotel-item"]')
    for card in propertyCards:
        try:  
            print("Extracted hotel count " + str(count))
            count+=1
            name = card.find_element_by_css_selector("h3.PropertyCard__HotelName").text
            
            # create new dict for first time entry
            if name not in hotels:
                hotels[name] = {}
            
            href = card.find_element_by_css_selector("a.PropertyCard__Link").get_attribute('href')
            star =  card.find_element_by_css_selector('i.ficon').get_attribute("title")
            price = card.find_element_by_css_selector("span.PropertyCardPrice__Value").text
            score = card.find_element_by_css_selector("span.sc-AxheI.kaXwHA").text
            reviews = card.find_element_by_css_selector("span.sc-AxheI.rbZVq.sc-AxirZ.fBDjam").text
            'create dict for each hotel'   
            hotels[name]['price'] = price
            hotels[name]['link'] = href
            hotels[name]['score'] = score
            hotels[name]['star'] = star
            hotels[name]['reviews'] = reviews

            init_price = card.find_element_by_css_selector("div.PropertyCardPrice--CrossOut").text
            hotels[name]['init_price'] = init_price
        except:
            print("Could not find something for hotel " + str(count))
    
    return hotels

def generate_check_in_and_out_dates():
    '''
    Generate list of tuples of check-in and check-out dates
    '''    
    check_in_and_out_dates = []
    for day in range(1,30):

        checkin = str(format(day,'02'))
        checkout = str(format(day+1,'02'))
        check_in_and_out_dates.append(("2020-11-"+checkin,"2020-11-"+checkout))

    # Add 1 more day, last day of Nov, checkout in 1 Dec 
    check_in_and_out_dates.append(['2020-11-30','2020,12-01'])
    return check_in_and_out_dates


def str_to_float(string):
    '''
    Used for pre-processing
    '''
    try:
        return float(re.sub('[^0-9.]','', string))
    except ValueError:
        return 0

In [ ]:
# Pre-processing steps

check_in_and_out_dates = generate_check_in_and_out_dates()

In [ ]:
# Load Selenium Driver
path = "D:\Downloads\chromedriver_win32\chromedriver.exe"
driver=webdriver.Chrome(path)

# Initialise hotels and dates Dictionary
hotels = {}
dates = {}

for date, x in check_in_and_out_dates:
    dates[date] = []
    

In [ ]:
# Scrapping logic

# Adult pax to search for
adults = 1

# Search through all days in the month
for check_in, check_out in check_in_and_out_dates:
    # enter URL
    driver.get(urlBuilder(check_in,check_out,adults))

    has_next_page = True
    page = 1
    first_page = True
    
    # Next page
    while has_next_page == True:
        print("Page : " + str(page))
        scrollToBtm()
        if first_page:
            # Scroll twice for first page
            driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
            scrollpgDwn(40)
            first_page = False
        hotels_next_day = extractHotelData()
        dates[check_in].extend(list(hotels_next_day.keys()))
        hotels.update(hotels_next_day)
        has_next_page = nextPage()
        page +=1


Page : 1
Extracted hotel count 0
Extracted hotel count 1
Extracted hotel count 2
Extracted hotel count 3
Could not find something for hotel 4
Extracted hotel count 4
Extracted hotel count 5
Extracted hotel count 6
Extracted hotel count 7
Extracted hotel count 8
Extracted hotel count 9
Extracted hotel count 10
Extracted hotel count 11
Extracted hotel count 12
Extracted hotel count 13
Could not find something for hotel 14
Extracted hotel count 14
Could not find something for hotel 15
Extracted hotel count 15
Extracted hotel count 16
Could not find something for hotel 17
Extracted hotel count 17
Extracted hotel count 18
Extracted hotel count 19
Extracted hotel count 20
Extracted hotel count 21
Extracted hotel count 22
Extracted hotel count 23
Could not find something for hotel 24
Extracted hotel count 24
Extracted hotel count 25
Extracted hotel count 26
Extracted hotel count 27
Extracted hotel count 28
Extracted hotel count 29
Extracted hotel count 30
Extracted hotel count 31
Extracted ho

Extracted hotel count 6
Could not find something for hotel 7
Extracted hotel count 7
Extracted hotel count 8
Extracted hotel count 9
Extracted hotel count 10
Extracted hotel count 11
Extracted hotel count 12
Extracted hotel count 13
Could not find something for hotel 14
Extracted hotel count 14
Could not find something for hotel 15
Extracted hotel count 15
Extracted hotel count 16
Extracted hotel count 17
Extracted hotel count 18
Extracted hotel count 19
Extracted hotel count 20
Extracted hotel count 21
Extracted hotel count 22
Extracted hotel count 23
Could not find something for hotel 24
Extracted hotel count 24
Extracted hotel count 25
Extracted hotel count 26
Extracted hotel count 27
Extracted hotel count 28
Extracted hotel count 29
Extracted hotel count 30
Extracted hotel count 31
Extracted hotel count 32
Extracted hotel count 33
Could not find something for hotel 34
Extracted hotel count 34
Extracted hotel count 35
Extracted hotel count 36
Extracted hotel count 37
Extracted hotel

Extracted hotel count 40
Extracted hotel count 41
Extracted hotel count 42
Extracted hotel count 43
Extracted hotel count 44
Extracted hotel count 45
Extracted hotel count 46
Extracted hotel count 47
Extracted hotel count 48
Page : 2
Extracted hotel count 0
Extracted hotel count 1
Extracted hotel count 2
Extracted hotel count 3
Could not find something for hotel 4
Extracted hotel count 4
Extracted hotel count 5
Extracted hotel count 6
Extracted hotel count 7
Extracted hotel count 8
Extracted hotel count 9
Extracted hotel count 10
Extracted hotel count 11
Could not find something for hotel 12
Extracted hotel count 12
Extracted hotel count 13
Could not find something for hotel 14
Extracted hotel count 14
Extracted hotel count 15
Extracted hotel count 16
Could not find something for hotel 17
Extracted hotel count 17
Extracted hotel count 18
Extracted hotel count 19
Extracted hotel count 20
Extracted hotel count 21
Extracted hotel count 22
Extracted hotel count 23
Could not find something 

Extracted hotel count 25
Extracted hotel count 26
Extracted hotel count 27
Extracted hotel count 28
Extracted hotel count 29
Extracted hotel count 30
Extracted hotel count 31
Could not find something for hotel 32
Extracted hotel count 32
Extracted hotel count 33
Could not find something for hotel 34
Extracted hotel count 34
Extracted hotel count 35
Extracted hotel count 36
Extracted hotel count 37
Extracted hotel count 38
Extracted hotel count 39
Extracted hotel count 40
Extracted hotel count 41
Extracted hotel count 42
Extracted hotel count 43
Could not find something for hotel 44
Extracted hotel count 44
Extracted hotel count 45
Extracted hotel count 46
Extracted hotel count 47
Extracted hotel count 48
Page : 3
Extracted hotel count 0
Extracted hotel count 1
Could not find something for hotel 2
Extracted hotel count 2
Extracted hotel count 3
Could not find something for hotel 4
Extracted hotel count 4
Extracted hotel count 5
Extracted hotel count 6
Extracted hotel count 7
Extracted h

Extracted hotel count 60
Extracted hotel count 61
Extracted hotel count 62
Could not find something for hotel 63
Extracted hotel count 63
Extracted hotel count 64
Extracted hotel count 65
Extracted hotel count 66
Extracted hotel count 67
Extracted hotel count 68
Extracted hotel count 69
Extracted hotel count 70
Extracted hotel count 71
Extracted hotel count 72
Could not find something for hotel 73
Extracted hotel count 73
Extracted hotel count 74
Extracted hotel count 75
Extracted hotel count 76
Extracted hotel count 77
Extracted hotel count 78
Extracted hotel count 79
Extracted hotel count 80
Extracted hotel count 81
Extracted hotel count 82
Could not find something for hotel 83
Extracted hotel count 83
Extracted hotel count 84
Extracted hotel count 85
Extracted hotel count 86
Extracted hotel count 87
Could not find something for hotel 88
Extracted hotel count 88
Extracted hotel count 89
Extracted hotel count 90
Extracted hotel count 91
Extracted hotel count 92
Extracted hotel count 9

...

Could not find something for hotel 4
Extracted hotel count 4
Extracted hotel count 5
Extracted hotel count 6
Extracted hotel count 7
Extracted hotel count 8
Extracted hotel count 9
Could not find something for hotel 10
Extracted hotel count 10
Extracted hotel count 11
Could not find something for hotel 12
Extracted hotel count 12
Extracted hotel count 13
Could not find something for hotel 14
Extracted hotel count 14
Extracted hotel count 15
Extracted hotel count 16
Extracted hotel count 17
Extracted hotel count 18
Extracted hotel count 19
Could not find something for hotel 20
Extracted hotel count 20
Extracted hotel count 21
Extracted hotel count 22
Extracted hotel count 23
Could not find something for hotel 24
Extracted hotel count 24
Extracted hotel count 25
Extracted hotel count 26
Could not find something for hotel 27
Extracted hotel count 27
Could not find something for hotel 28
Extracted hotel count 28
Extracted hotel count 29
Extracted hotel count 30
Extracted hotel count 31
Cou

Extracted hotel count 49
Extracted hotel count 50
Extracted hotel count 51
Extracted hotel count 52
Could not find something for hotel 53
Extracted hotel count 53
Extracted hotel count 54
Could not find something for hotel 55
Extracted hotel count 55
Could not find something for hotel 56
Extracted hotel count 56
Extracted hotel count 57
Extracted hotel count 58
Could not find something for hotel 59
Extracted hotel count 59
Extracted hotel count 60
Could not find something for hotel 61
Extracted hotel count 61
Extracted hotel count 62
Could not find something for hotel 63
Extracted hotel count 63
Extracted hotel count 64
Extracted hotel count 65
Extracted hotel count 66
Extracted hotel count 67
Extracted hotel count 68
Extracted hotel count 69
Extracted hotel count 70
Extracted hotel count 71
Could not find something for hotel 72
Extracted hotel count 72
Could not find something for hotel 73
Extracted hotel count 73
Extracted hotel count 74
Extracted hotel count 75
Extracted hotel count

Extracted hotel count 47
Extracted hotel count 48
Page : 2
Extracted hotel count 0
Extracted hotel count 1
Could not find something for hotel 2
Extracted hotel count 2
Extracted hotel count 3
Could not find something for hotel 4
Extracted hotel count 4
Extracted hotel count 5
Could not find something for hotel 6
Extracted hotel count 6
Extracted hotel count 7
Extracted hotel count 8
Extracted hotel count 9
Extracted hotel count 10
Extracted hotel count 11
Extracted hotel count 12
Extracted hotel count 13
Could not find something for hotel 14
Extracted hotel count 14
Extracted hotel count 15
Extracted hotel count 16
Extracted hotel count 17
Could not find something for hotel 18
Extracted hotel count 18
Extracted hotel count 19
Extracted hotel count 20
Extracted hotel count 21
Extracted hotel count 22
Extracted hotel count 23
Could not find something for hotel 24
Extracted hotel count 24
Could not find something for hotel 25
Extracted hotel count 25
Extracted hotel count 26
Extracted hot

Could not find something for hotel 41
Extracted hotel count 41
Page : 1
Extracted hotel count 0
Extracted hotel count 1
Extracted hotel count 2
Extracted hotel count 3
Could not find something for hotel 4
Extracted hotel count 4
Extracted hotel count 5
Could not find something for hotel 6
Extracted hotel count 6
Could not find something for hotel 7
Extracted hotel count 7
Extracted hotel count 8
Extracted hotel count 9
Could not find something for hotel 10
Extracted hotel count 10
Extracted hotel count 11
Extracted hotel count 12
Extracted hotel count 13
Could not find something for hotel 14
Extracted hotel count 14
Could not find something for hotel 15
Extracted hotel count 15
Extracted hotel count 16
Could not find something for hotel 17
Extracted hotel count 17
Extracted hotel count 18
Extracted hotel count 19
Extracted hotel count 20
Extracted hotel count 21
Extracted hotel count 22
Extracted hotel count 23
Could not find something for hotel 24
Extracted hotel count 24
Extracted ho

Extracted hotel count 43
Could not find something for hotel 44
Extracted hotel count 44
Could not find something for hotel 45
Extracted hotel count 45
Could not find something for hotel 46
Extracted hotel count 46
Could not find something for hotel 47
Extracted hotel count 47
Could not find something for hotel 48
Extracted hotel count 48
Could not find something for hotel 49


In [ ]:
len(hotels)

345

In [ ]:
import pandas as pd
df = pd.DataFrame(hotels)
df = df.transpose()

In [ ]:
# Exporting Dirty csv
df.to_csv ('dataframe_1adult.csv', index = True, header=True)

In [ ]:
# Data cleaning
dfclean = df.copy()

dfclean = dfclean.dropna()
dfclean['init_price'] = dfclean['init_price'].apply(lambda x: int(re.sub('[^0-9]','', x)) )
dfclean['price'] = dfclean['price'].apply(lambda x: int(re.sub('[^0-9]','', x)) )
dfclean['reviews'] = dfclean['reviews'].apply(lambda x: int(re.sub('[^0-9]','', x)) )
dfclean['score'] = dfclean['score'].apply(str_to_float)
dfclean['star'] = dfclean['star'].apply(str_to_float)
dfclean['short_name'] = dfclean['link'].apply(lambda x: x.split("/")[4])

In [ ]:
dfclean.head()

,init_price,link,price,reviews,score,star,short_name
Amara Sanctuary Resort Sentosa (SG Clean Certified),917,https://www.agoda.com/en-us/amara-sanctuary-re...,306,7209,7.6,5.0,amara-sanctuary-resort-sentosa
W Singapore - Sentosa Cove (SG Clean Certified),913,https://www.agoda.com/en-us/w-singapore-sentos...,599,2974,8.6,0.0,w-singapore-sentosa-cove
Hotel Boss (SG Clean Certified),323,https://www.agoda.com/en-us/hotel-boss/hotel/s...,95,13993,7.8,0.0,hotel-boss
"Capri by Fraser - Changi City (SG Clean Certified,Staycation approved)",500,https://www.agoda.com/en-us/capri-by-fraser-ch...,145,6787,8.6,0.0,capri-by-fraser-changi-city
Hotel G Singapore (SG Clean Certified),349,https://www.agoda.com/en-us/hotel-g-singapore/...,79,8095,8.4,0.0,hotel-g-singapore


In [ ]:
# Exporting data to csv
dfclean.to_csv ('dfclean_1adult.csv', index = True, header=True)
(pd.DataFrame.from_dict(data=dates, orient='index')
   .to_csv('days_1adults.csv', header=False))